# ECG Reviewer

## Install Dependencies

In [ ]:
# Uncomment and use instead if installing the Python dependencies from a Google Cloud Storage Bucket
# !gsutil cp -r gs://paws-dev.starr-data.us .
# !tar -xzf packages.tar.gz
# !pip install -q --no-index --find-links=packages spezi-data-pipeline ipywidgets

!pip install -r requirements.txt

## Import Dependencies & Code

In [ ]:
from firebase_admin import firestore
from spezi_data_pipeline.data_access.firebase_fhir_data_access import FirebaseFHIRAccess
from spezi_data_pipeline.data_flattening.fhir_resources_flattener import flatten_fhir_resources
from modules.visualization import ECGDataViewer
from modules.utils import process_ecg_data, export_database_in_csv

## Connect to Firebase and download ECG data

In [ ]:
# Define the Firestore client instance
db = firestore.Client()

# Define the collection name where your FHIR observations are stored and the input code if filtering is needed
collection_name = "users"
subcollection_name = "HealthKit"
loinc_codes = ["131328"]

# Connect to Firestore
firebase_access = FirebaseFHIRAccess(db=db)

# Optional, if db is not provided
firebase_access.connect()

# Download and process data
fhir_observations = firebase_access.fetch_data(collection_name, subcollection_name, loinc_codes)
flattened_fhir_dataframe = flatten_fhir_resources(fhir_observations)

if not flattened_fhir_dataframe.df.empty:
    ecg_data = process_ecg_data(firebase_access.db, flattened_fhir_dataframe.df)

### Preview the ECG data

In [ ]:
ecg_data.head()

### Export the database in .CSV format

In [ ]:
export_database_in_csv(ecg_data)

## Review ECG recordings

In [ ]:
reviewer = ECGDataViewer(ecg_data, firebase_access.db)